In [ ]:
import os
import pandas as pd
import requests
import shutil

for csv_file in os.listdir('data')[28:]:
    print(f'{csv_file} 다운로드중......')
    if csv_file.endswith('.csv'):
        df = pd.read_csv(os.path.join('data', csv_file))
        
        for idx, row in df.iterrows():
            if idx >= 1000:
                break
            image_url = row['product_url']
            file_name = row['bottom_category']
            image_name = f"{file_name}_{idx}.jpg"
            image_path = os.path.join('image', image_name)
            
            try:
                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(image_path, 'wb') as out_file:
                        shutil.copyfileobj(response.raw, out_file)
                else:
                    print(f"이미지를 다운로드하는데 실패 {image_url}")
                    continue
            except Exception as e:
                print(f"이미지를 다운로드하는 도중 예외가 발생 {e}")
                continue
            finally:
                del response



In [21]:
import requests

ip = '15.165.147.231'

# 회원가입 post
signup_url = f'http://{ip}/api/accounts/signup/'
data = {
    'username' : 'test123',
    'password' : '123456123456', # 10자이상
    'phone_number' : '123456123456',
    'nickname' : '놀부심보',
    'email' : 'test123@ejuhh3.io', # emailfield
}
res = requests.post(signup_url, data=data)

res.status_code
res.json()


# 로그인
login_url = f'http://{ip}/api/accounts/login/'
data = {
    'email': 'test123@ejuhh3.io',
    'password': '123456123456'
}
res = requests.post(login_url, data=data)
print(res.status_code)
ac_token = res.json()['access']

200


In [ ]:
import os
import pandas as pd
import requests

# 데이터 디렉토리 및 API 설정
data_dir = 'data'
image_dir = 'image'
upload_url = f"http://{ip}/api/images/upload/product/"
article_url = f"http://{ip}/api/articles/"
headers = {'Authorization': f'Bearer {ac_token}'}

for csv_file in os.listdir(data_dir)[18:]:
    if csv_file.endswith('.csv'):
        df = pd.read_csv(os.path.join(data_dir, csv_file))
        
        for image_file in os.listdir(image_dir)[18541:]:
            try:
                bottom_category, num_str = image_file.rsplit('_', 1)
                num = int(num_str.split('.')[0])  # 이 부분을 문자열로 변환하여 처리
                
                if num < len(df) and df.iloc[num]['bottom_category'] == bottom_category:
                    row = df.iloc[num]
                    
                    file_path = os.path.join(image_dir, image_file)
                    
                    with open(file_path, 'rb') as img_file:
                        files = {'images': img_file}
                        response = requests.post(upload_url, headers=headers, files=files)
                        # print(response.status_code)
                    
                    if response.status_code != 201:
                        print(f"이미지 업로드 실패: {file_path} - 상태 코드: {response.status_code}")
                        continue
                    
                    
                    # print(response.json())
                    uploaded_data = response.json()
                    if 'uploaded_images' in uploaded_data:
                        image_urls = [img['image_url'] for img in uploaded_data['uploaded_images']]
                        # print(image_urls)
                    else:
                        print("업로드된 이미지 정보를 가져올 수 없습니다.")
                        continue
                    
                    # 게시물 작성
                    content = f"""
                    {row['title']}
                    
                    사이즈 {row['size']}, 실측사이즈 어깨 45cm / 가슴폭 53 / 팔 23 / 기장 72
                    
                    색상 {row['color']} 스트라이프, 얇고 가벼우며 시원한 재질의 {row['bottom_category']}입니다.
                    
                    신축성이 좋아 착용감이 좋고 활동성이 우수합니다.
                    
                    {row['status']}이며 잘 관리되어 입기 좋습니다.
                    """
                    
                    data = {
                        "title": row['title'],
                        "content": content,
                        "product": {
                            "option": {"size": row['size'], "color": row['color']},
                            "category": {"top_category": row['top_category'], "bottom_category": row['bottom_category']},
                            "price": int(row['price']),
                            "product_type": 0,
                            'product_status': row['status'],
                            "image_urls": image_urls
                        }
                    }
                    
                    response = requests.post(article_url, headers=headers, json=data)
                    
                    if response.status_code == 201:
                        print('성공')
                    else:
                        print(f"게시물 생성 실패: {data['title']} - 상태 코드: {response.status_code}")
            except Exception as e:
                print(f"오류 발생: {e} - 파일: {image_file}")


        

In [19]:
image_dir = 'image'
for i, image_file in enumerate(os.listdir(image_dir)):
    if image_file == '베스트_724.jpg':
        print(i)



18541
